# TITLE

We downloaded the files from: <br>
accidentalidad.csv: http://medata.gov.co/dataset/accidentalidad 
<br>
AccidentalidadgeoreferenciadaYYYY:
<br>
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2019
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2018
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2017
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2016
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2015
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2014
<br>

To ingest about dataset we decided to use two different dataframes, accidentalidad dataframe ```a_df``` to aggregate the differents files by year and include the incident time wich is not included in the accidentalidad_georrererenciada dataframe ```ag_df```, so we use these two dataframes to verify the integrity of the datasets and build one dataset with all the information we need.

In the following code we clean and merge the datasets:

# Download all the data


In [299]:
import requests
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from shapely import geometry
import numpy as np
from ast import literal_eval
from unidecode import unidecode
import holidays
import random
pd.options.display.max_columns = None


#!brew install spatialindex #needs rtree to work
#pip install rtree


In [300]:
# url_incidentes = 'https://ds4a.blob.core.windows.net/ds4a/Incidentes2014_2020.xlsx'

In [301]:
# r =requests.get(url_incidentes, allow_redirects=True)
# open('./data/Incidentes2014_2020.xlsx', 'wb').write(r.content)

In [302]:
i_df = pd.read_excel('./data/Incidentes2014_2020.xlsx')

In [303]:
i_df.shape

(288983, 15)

In [304]:
i_df.columns

Index(['NRO_RADICADO', 'Latitud', 'Longitud', 'CLASE_ACCIDENTE', 'DIRECCION',
       'CBML', 'Gravedad Incidente', 'nombre comuna', 'Barrio', 'Diseño',
       'Año', 'FECHA_ACCIDENTE', 'HORA_ACCIDENTE', 'codigo comuna', 'Comuna'],
      dtype='object')

In [305]:
i_df.drop(columns=['Comuna'],inplace=True)

In [306]:
i_df.rename(columns={"NRO_RADICADO": "Radicado", "CLASE_ACCIDENTE":"Type", "DIRECCION":"Address", "CBML":"Cbml", "Barrio":"Borough", "nombre comuna":"Comuna", "codigo comuna":"codigo_comuna", "Diseño":"Desing", "Longitud":"Longitude", "Latitud":"Latitude",  "Gravedad Incidente":"Severity", "FECHA_ACCIDENTE":"Date", "Año":"Year", "HORA_ACCIDENTE":"Time"},inplace=True)
i_df.co

,Radicado,Latitude,Longitude,Type,Address,Cbml,Severity,Comuna,Borough,Desing,Year,Date,Time,codigo_comuna
0,1423828,6.219016,-75.602726,Choque,CR 80 CL 8,1611,Con heridos,Belén,Loma de los Bernal,Tramo de via,2014,2014-01-01,02:20:00,16
1,1423839,6.260009,-75.568185,Atropello,CR 53 CL 61,1003,Con heridos,La Candelaria,Jesús Nazareno,Tramo de via,2014,2014-01-01,00:50:00,10
2,1423840,6.264765,-75.549943,Atropello,CR 39 CL 70,0308,Con heridos,Manrique,Manrique Oriental,Tramo de via,2014,2014-01-01,01:00:00,3
3,1423849,6.234327,-75.607611,Atropello,CL 32 CR 84,1617,Con heridos,Belén,Las Mercedes,Tramo de via,2014,2014-01-01,00:37:00,16
4,1423890,6.300243,-75.579644,Caida Ocupante,CR 80 CL 101,0603,Con heridos,Doce De Octubre,Doce de Octubre No.2,Tramo de via,2014,2014-01-01,10:40:00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288978,1717678,6.220716,-75.559017,Caida Ocupante,CL 19A CR 27,9004,Con heridos,Corregimiento De Santa Elena,9004,Lote o Predio,2020,2020-06-30,21:00:00,90
288979,1717650,6.286417,-75.570338,Choque,CR 65 CL 94,0511,Con heridos,Castilla,Castilla,Interseccion,2020,2020-06-30,21:05:00,5
288980,1717526,6.221415,-75.703776,Choque,CR 86 CL 44 B Sur,1204,Con heridos,La América,La América,Tramo de via,2020,2020-06-30,21:15:00,12
288981,1717545,6.235761,-75.538379,Otro,CR 12 CL 53,0813,Con heridos,Villa Hermosa,Villatina,Lote o Predio,2020,2020-06-30,22:00:00,8


In [422]:
i_df.codigo_comuna.value_counts()

10         55481
11         28867
5          26545
14         23216
Sin Inf    21976
15         19706
16         17856
7          17656
4          15405
9          10447
12          8588
3           8278
6           7389
8           7120
13          4636
80          4342
1           3931
2           3683
60          2640
90           605
70           599
50            18
Name: codigo_comuna, dtype: int64

In [308]:
for i in i_df.isna().any()[i_df.isna().any() == True].index:
    print(i,' : ' ,i_df[i_df[i].isna()][i].size,' NaN')

Radicado  :  3  NaN
Latitude  :  534  NaN
Longitude  :  534  NaN
Type  :  7  NaN
Cbml  :  22600  NaN
Borough  :  23472  NaN
Desing  :  785  NaN
Time  :  2  NaN


In [309]:
i_df.loc[(i_df.Radicado.isna()),'Radicado'] = i_df[i_df.Radicado.isna()].Radicado.apply(lambda x :str(int(round(random.random() * 9999999,0))))
i_df.Radicado.value_counts()

1627001    1
1628092    1
1486421    1
1614731    1
1706457    1
          ..
1462162    1
1462161    1
1476701    1
1462159    1
1572864    1
Name: Radicado, Length: 288983, dtype: int64

In [333]:
for i in i_df.isna().any()[i_df.isna().any() == True].index:
    print(i,' : ' ,i_df[i_df[i].isna()][i].size,' NaN')

Latitude  :  534  NaN
Longitude  :  534  NaN
Type  :  7  NaN
Cbml_standard  :  22600  NaN
Borough_original  :  23472  NaN
Desing  :  785  NaN
Time  :  2  NaN
Borough_standard  :  24331  NaN
Comuna_standard  :  24331  NaN
Cbml  :  23450  NaN
Borough  :  23450  NaN
Comuna  :  23450  NaN


In [363]:
url_postal_codes_with_boroughs = 'https://ds4a.blob.core.windows.net/ds4a/postalCodesBoroughs.json'
pcb_df = pd.read_json(url_postal_codes_with_boroughs, dtype=str)

In [364]:
pcb_df

,postalCode,state,city,locType,boroughs,otherBoroughs
0,055417,ANTIOQUIA,ITAGÜI,Rural,Sin Informacion de Barrios,"El Jardín, La Verde"
1,055420,ANTIOQUIA,ENVIGADO,Urbano,"Alto de Misael, El Chocho, El Esmeraldal, La I...",Sin Informacion de Veredas
2,055422,ANTIOQUIA,ENVIGADO,Urbano,"Alcalá, Bosques de Zúñiga, Bucarest, Centro, E...",Sin Informacion de Veredas
3,055428,ANTIOQUIA,ENVIGADO,Rural,Sin Informacion de Barrios,Sin Informacion de Veredas
4,050048,ANTIOQUIA,MEDELLIN,Rural,Sin Informacion de Barrios,"La Aldea, La Esmeralda, La Frisola, La Sucia..."
5,050029,ANTIOQUIA,MEDELLIN,Rural,Sin Informacion de Barrios,"Área de Expansión San Antonio de Prado, Buga P..."
6,050027,ANTIOQUIA,MEDELLIN,Rural,Sin Informacion de Barrios,Aguas Frías
7,050047,ANTIOQUIA,MEDELLIN,Rural,"Monteclaro, Pajarito",Sin Informacion de Veredas
8,050028,ANTIOQUIA,MEDELLIN,Rural,Sin Informacion de Barrios,Sin Informacion de Veredas
9,050007,ANTIOQUIA,MEDELLIN,Rural,Sin Informacion de Barrios,"Barro Blanco, La Honda"


In [334]:
url_postal_codes = "https://ds4a.blob.core.windows.net/ds4a/MedPostalCode.geojson"
pc_df = gpd.read_file(url_postal_codes)

In [335]:
pc_df

,zonapostal,objectid,shapearea,codigo_pos,shapelen,geometry
0,0554,43,6680212.97223428,055417,15610.854268663901,"MULTIPOLYGON (((-75.60091 6.19783, -75.60101 6..."
1,0554,44,5685948.57423864,055420,10608.2667799046,"MULTIPOLYGON (((-75.55891 6.17570, -75.55888 6..."
2,0554,45,3859621.32287542,055422,9744.954672637381,"MULTIPOLYGON (((-75.57954 6.18353, -75.57959 6..."
3,0554,46,32037902.1635823,055428,30817.9220505459,"MULTIPOLYGON (((-75.48371 6.19832, -75.48374 6..."
4,0500,47,58662193.604592,050048,38858.1197240499,"MULTIPOLYGON (((-75.66876 6.37313, -75.66862 6..."
5,0500,48,56699770.0318451,050029,38135.547696996604,"MULTIPOLYGON (((-75.66451 6.24257, -75.66306 6..."
6,0500,49,14426034.1282442,050027,23560.232614092598,"MULTIPOLYGON (((-75.61514 6.22775, -75.61500 6..."
7,0500,50,25451699.0709864,050047,31060.6559913341,"MULTIPOLYGON (((-75.64461 6.32579, -75.64312 6..."
8,0500,51,7430001.85045064,050028,15941.2406602466,"MULTIPOLYGON (((-75.62185 6.24124, -75.62169 6..."
9,0500,52,27791286.0451787,050007,32687.336654219303,"MULTIPOLYGON (((-75.51747 6.29918, -75.51699 6..."


In [311]:
url = "https://ds4a.blob.core.windows.net/ds4a/Barrios.geojson"
borough_df = gpd.read_file(url)


In [312]:
borough_df.rename(columns={'NOMBRE_COMUNA':'Comuna','CODIGO': 'Cbml','NOMBRE_BARRIO':'Borough' },inplace=True)

In [313]:
borough_df.drop(columns=['OBJECTID', 'SHAPEAREA', 'SHAPELEN', 'COMUNA', 'BARRIO'],inplace=True)

In [314]:
b_df = borough_df[['Cbml', 'Borough', 'Comuna']].copy()
b_df

,Cbml,Borough,Comuna
0,0901,Juan Pablo II,BUENOS AIRES
1,1409,El Tesoro,EL POBLADO
2,1211,Simón Bolívar,LA AMERICA
3,0604,Pedregal,DOCE DE OCTUBRE
4,9008,Piedras Blancas,SANTA ELENA
...,...,...,...
345,0202,Playón de Los Comuneros,SANTA CRUZ
346,1112,El Velódromo,LAURELES
347,1619,Miravalle,BELEN
348,0303,Campo Valdés No. 2,MANRIQUE


In [315]:
i_df = i_df.merge(b_df,how='left', on='Cbml', suffixes=('_original','')).drop_duplicates()


In [316]:
i_df

,Radicado,Latitude,Longitude,Type,Address,Cbml,Severity,Comuna_original,Borough_original,Desing,Year,Date,Time,codigo_comuna,Borough,Comuna
0,1423828,6.219016,-75.602726,Choque,CR 80 CL 8,1611,Con heridos,Belén,Loma de los Bernal,Tramo de via,2014,2014-01-01,02:20:00,16,Loma de los Bernal,BELEN
1,1423839,6.260009,-75.568185,Atropello,CR 53 CL 61,1003,Con heridos,La Candelaria,Jesús Nazareno,Tramo de via,2014,2014-01-01,00:50:00,10,Jesús Nazareno,LA CANDELARIA
2,1423840,6.264765,-75.549943,Atropello,CR 39 CL 70,0308,Con heridos,Manrique,Manrique Oriental,Tramo de via,2014,2014-01-01,01:00:00,3,Manrique Oriental,MANRIQUE
3,1423849,6.234327,-75.607611,Atropello,CL 32 CR 84,1617,Con heridos,Belén,Las Mercedes,Tramo de via,2014,2014-01-01,00:37:00,16,Las Mercedes,BELEN
4,1423890,6.300243,-75.579644,Caida Ocupante,CR 80 CL 101,0603,Con heridos,Doce De Octubre,Doce de Octubre No.2,Tramo de via,2014,2014-01-01,10:40:00,6,Doce de Octubre No.2,DOCE DE OCTUBRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289094,1717678,6.220716,-75.559017,Caida Ocupante,CL 19A CR 27,9004,Con heridos,Corregimiento De Santa Elena,9004,Lote o Predio,2020,2020-06-30,21:00:00,90,Las Palmas,SANTA ELENA
289095,1717650,6.286417,-75.570338,Choque,CR 65 CL 94,0511,Con heridos,Castilla,Castilla,Interseccion,2020,2020-06-30,21:05:00,5,Castilla,CASTILLA
289096,1717526,6.221415,-75.703776,Choque,CR 86 CL 44 B Sur,1204,Con heridos,La América,La América,Tramo de via,2020,2020-06-30,21:15:00,12,La América,LA AMERICA
289097,1717545,6.235761,-75.538379,Otro,CR 12 CL 53,0813,Con heridos,Villa Hermosa,Villatina,Lote o Predio,2020,2020-06-30,22:00:00,8,Villatina,VILLA HERMOSA


In [317]:

temp_i_df = i_df[(~i_df.Latitude.isna())][['Radicado','Latitude', 'Longitude']]

i_gdf = gpd.GeoDataFrame(
    temp_i_df, geometry=gpd.points_from_xy(temp_i_df.Longitude, temp_i_df.Latitude))[['Radicado','geometry']]
i_gdf

,Radicado,geometry
0,1423828,POINT (-75.60273 6.21902)
1,1423839,POINT (-75.56818 6.26001)
2,1423840,POINT (-75.54994 6.26476)
3,1423849,POINT (-75.60761 6.23433)
4,1423890,POINT (-75.57964 6.30024)
...,...,...
289094,1717678,POINT (-75.55902 6.22072)
289095,1717650,POINT (-75.57034 6.28642)
289096,1717526,POINT (-75.70378 6.22142)
289097,1717545,POINT (-75.53838 6.23576)


In [318]:
i_gdf = gpd.sjoin(i_gdf, borough_df,op='within', how='left').drop_duplicates()
i_gdf

,Radicado,geometry,index_right,Cbml,Borough,Comuna
0,1423828,POINT (-75.60273 6.21902),303.0,1611,Loma de los Bernal,BELEN
1,1423839,POINT (-75.56818 6.26001),11.0,1003,Jesús Nazareno,LA CANDELARIA
2,1423840,POINT (-75.54994 6.26476),31.0,0308,Manrique Oriental,MANRIQUE
3,1423849,POINT (-75.60761 6.23433),190.0,1617,Las Mercedes,BELEN
4,1423890,POINT (-75.57964 6.30024),53.0,0603,Doce de Octubre No.2,DOCE DE OCTUBRE
...,...,...,...,...,...,...
289094,1717678,POINT (-75.55902 6.22072),83.0,0916,Asomadera No. 3,BUENOS AIRES
289095,1717650,POINT (-75.57034 6.28642),203.0,0511,Castilla,CASTILLA
289096,1717526,POINT (-75.70378 6.22142),NaN,NaN,NaN,NaN
289097,1717545,POINT (-75.53838 6.23576),13.0,0813,Villatina,VILLA HERMOSA


In [319]:
i_gdf.dropna(inplace=True)

In [320]:
i_gdf.drop(columns=['geometry','index_right'],inplace=True)

In [323]:
i_df = i_df.merge(i_gdf, on='Radicado', how='left', suffixes=('_standard', '')).drop_duplicates()

# i_df.join(i_gdf.set_index('Radicado'), on='Radicado', how='left', lsuffix='_old').drop_duplicates()


In [390]:
i_df

,Radicado,Latitude,Longitude,Type,Address,Cbml_standard,Severity,Comuna_original,Borough_original,Desing,Year,Date,Time,codigo_comuna,Borough_standard,Comuna_standard,Cbml,Borough,Comuna
0,1423828,6.219016,-75.602726,Choque,CR 80 CL 8,1611,Con heridos,Belén,Loma de los Bernal,Tramo de via,2014,2014-01-01,02:20:00,16,Loma de los Bernal,BELEN,1611,Loma de los Bernal,BELEN
1,1423839,6.260009,-75.568185,Atropello,CR 53 CL 61,1003,Con heridos,La Candelaria,Jesús Nazareno,Tramo de via,2014,2014-01-01,00:50:00,10,Jesús Nazareno,LA CANDELARIA,1003,Jesús Nazareno,LA CANDELARIA
2,1423840,6.264765,-75.549943,Atropello,CR 39 CL 70,0308,Con heridos,Manrique,Manrique Oriental,Tramo de via,2014,2014-01-01,01:00:00,3,Manrique Oriental,MANRIQUE,0308,Manrique Oriental,MANRIQUE
3,1423849,6.234327,-75.607611,Atropello,CL 32 CR 84,1617,Con heridos,Belén,Las Mercedes,Tramo de via,2014,2014-01-01,00:37:00,16,Las Mercedes,BELEN,1617,Las Mercedes,BELEN
4,1423890,6.300243,-75.579644,Caida Ocupante,CR 80 CL 101,0603,Con heridos,Doce De Octubre,Doce de Octubre No.2,Tramo de via,2014,2014-01-01,10:40:00,6,Doce de Octubre No.2,DOCE DE OCTUBRE,0603,Doce de Octubre No.2,DOCE DE OCTUBRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288979,1717678,6.220716,-75.559017,Caida Ocupante,CL 19A CR 27,9004,Con heridos,Corregimiento De Santa Elena,9004,Lote o Predio,2020,2020-06-30,21:00:00,90,Las Palmas,SANTA ELENA,0916,Asomadera No. 3,BUENOS AIRES
288980,1717650,6.286417,-75.570338,Choque,CR 65 CL 94,0511,Con heridos,Castilla,Castilla,Interseccion,2020,2020-06-30,21:05:00,5,Castilla,CASTILLA,0511,Castilla,CASTILLA
288981,1717526,6.221415,-75.703776,Choque,CR 86 CL 44 B Sur,1204,Con heridos,La América,La América,Tramo de via,2020,2020-06-30,21:15:00,12,La América,LA AMERICA,NaN,NaN,NaN
288982,1717545,6.235761,-75.538379,Otro,CR 12 CL 53,0813,Con heridos,Villa Hermosa,Villatina,Lote o Predio,2020,2020-06-30,22:00:00,8,Villatina,VILLA HERMOSA,0813,Villatina,VILLA HERMOSA


In [418]:
# import geocoder
# import json
# to_clean = i_df[i_df.Cbml != i_df.Cbml_standard].copy()

# batch = [[x,y] for x, y in zip(to_clean.Latitude,to_clean.Longitude) ]
# len(batch)
g = geocoder.bing([[40.7943, -73.970859], [48.845580, 2.321807]], method='batch_reverse',key='An8rGhIvWo0WthZ813a1cKK7gUjQ_6c2OFAlqLvi3pP9El8P0s_k7beCQEW2EzR1')
for x in g:
    print(x)
# with open('batch_reverse.json', 'w') as outfile:
#     json.dump(g, outfile, ensure_ascii=False, indent=4)

[329 W 96th St, New York, NY 10025]
[114B Rue de Vaugirard, 75006 Paris]


In [421]:
to_clean.shape

(69676, 19)

In [431]:
from geopy.geocoders import Bing
from geopy.distance import distance
g = Bing('An8rGhIvWo0WthZ813a1cKK7gUjQ_6c2OFAlqLvi3pP9El8P0s_k7beCQEW2EzR1')

to_clean = i_df[i_df.Cbml != i_df.Cbml_standard].copy()
collistFromAddress =['postalCodeFromAddress','stateFromAddress','cityFromAddress','locTypeFromAddress','boroughsFromAddress','otherBoroughsFromAddress']
for col in collistFromAddress:
    to_clean[col] = ''

collistFromCoord =['postalCodeFromCoord','stateFromCoord','cityFromCoord','locTypeFromCoord','boroughsFromCoord','otherBoroughsFromCoord']
for col in collistFromCoord:
    to_clean[col] = ''
currentIndex = 1
oks = 0
for i, row in to_clean.iterrows():
    loc_from_address = g.geocode(row.Address+',Medellin,Colombia')
    loc_from_coord = g.reverse(str(row.Latitude)+','+str(row.Longitude))

    if 'raw' in loc_from_address:
        pc_from_address = loc_from_address.raw['address']['postalCode'] if 'postalCode' in loc_from_address.raw['address'] else ''
        if pc_from_address != '' and not pcb_df[pcb_df.postalCode==pc_from_address].empty:
            to_clean.loc[(to_clean.Radicado == row.Radicado),collistFromAddress] = pcb_df[pcb_df.postalCode==pc_from_address].values 
            to_clean.loc[to_clean.Radicado == row.Radicado,'PostalCodeFromAddress'] =  pc_from_address
        to_clean.loc[to_clean.Radicado == row.Radicado,'AddressFromAddress'] =  loc_from_address.raw['name'] 
        to_clean.loc[to_clean.Radicado == row.Radicado,'ConfidenceFromAddress'] =  loc_from_address.raw['confidence']
        to_clean.loc[to_clean.Radicado == row.Radicado,'LocalityFromAddress'] =  loc_from_address.raw['address']['locality'] 
        to_clean.loc[to_clean.Radicado == row.Radicado,'MatchCodesFromAddress'] =  ','.join(loc_from_address.raw['matchCodes'])

    p_from_address = (loc_from_address.latitude, loc_from_address.longitude)
    d_from_address = distance(p_from_address, p_from_coord).km
    to_clean.loc[to_clean.Radicado == row.Radicado,'DistanceFromAddress'] =  d_from_address
    
    if 'raw' in loc_from_coord:
        pc_from_coord = loc_from_coord.raw['address']['postalCode'] if 'postalCode' in loc_from_coord.raw['address'] else ''
        if pc_from_coord != '' and not pcb_df[pcb_df.postalCode==pc_from_coord].empty:
            to_clean.loc[to_clean.Radicado == row.Radicado,collistFromCoord] = pcb_df[pcb_df.postalCode==pc_from_coord].values
            to_clean.loc[to_clean.Radicado == row.Radicado,'PostalCodeFromCoord'] =  pc_from_coord
        to_clean.loc[to_clean.Radicado == row.Radicado,'AddressFromCoord'] =  loc_from_coord.raw['name'] 
        to_clean.loc[to_clean.Radicado == row.Radicado,'ConfidenceFromCoord'] =  loc_from_coord.raw['confidence']
        to_clean.loc[to_clean.Radicado == row.Radicado,'LocalityFromCoord'] =  loc_from_coord.raw['address']['locality'] 
        to_clean.loc[to_clean.Radicado == row.Radicado,'MatchCodesFromCoord'] =  ','.join(loc_from_coord.raw['matchCodes'])
       

    p_from_coord = (row.Latitude, row.Longitude)
    d_from_coord = distance((loc_from_coord.latitude,loc_from_coord.longitude), p_from_coord).km
    to_clean.loc[to_clean.Radicado == row.Radicado,'DistanceFromCoord'] =  d_from_coord
    
    if d_from_address < 1:
        oks = oks + 1
        print(oks,currentIndex)
        print(round(oks/currentIndex,2)*100,'%')
        print(to_clean.loc[to_clean.Radicado == row.Radicado].Radicado.values)
        to_clean.loc[to_clean.Radicado == row.Radicado ,'DistanceAnalysis'] =  'Ok'
    else:
        to_clean.loc[to_clean.Radicado == row.Radicado ,'DistanceAnalysis'] = 'Nok'
    currentIndex = currentIndex + 1 
to_clean

1 58
2.0 %
[1424094]
2 69
3.0 %
[1424219]
3 72
4.0 %
[1424224]
4 78
5.0 %
[1424243]
5 83
6.0 %
[1425217]
6 232
3.0 %
[1424875]
7 257
3.0 %
[1425010]
8 323
2.0 %
[1425128]
9 331
3.0 %
[1425168]
10 379
3.0 %
[1425483]
11 382
3.0 %
[1425490]
12 384
3.0 %
[1425453]
13 411
3.0 %
[1425707]
14 438
3.0 %
[1425827]
15 505
3.0 %
[1426106]
16 538
3.0 %
[1426213]
17 617
3.0 %
[1426653]
18 630
3.0 %
[1426589]
19 672
3.0 %
[1426797]
20 691
3.0 %
[1426828]
21 728
3.0 %
[1427027]
22 774
3.0 %
[1427324]
23 777
3.0 %
[1427333]
24 824
3.0 %
[1427488]
25 854
3.0 %
[1427659]
26 873
3.0 %
[1427768]
27 882
3.0 %
[1427891]
28 911
3.0 %
[1427888]
29 966
3.0 %
[1428203]
30 1018
3.0 %
[1428369]
31 1068
3.0 %
[1428761]
32 1088
3.0 %
[1428861]
33 1113
3.0 %
[1428874]
34 1166
3.0 %
[1429218]
35 1173
3.0 %
[1429478]
36 1181
3.0 %
[1429162]
37 1229
3.0 %
[1429605]
38 1266
3.0 %
[1429553]
39 1296
3.0 %
[1429735]
40 1348
3.0 %
[1429981]
41 1385
3.0 %
[1430130]
42 1440
3.0 %
[1430413]
43 1597
3.0 %
[1431042]
44 1688
3.0

GeocoderUnavailable: HTTPSConnectionPool(host='dev.virtualearth.net', port=443): Max retries exceeded with url: /REST/v1/Locations?query=Tramo+9+Kilometro+999%2CMedellin%2CColombia&key=An8rGhIvWo0WthZ813a1cKK7gUjQ_6c2OFAlqLvi3pP9El8P0s_k7beCQEW2EzR1&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x18c2581d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
to_clean[to_clean.DistanceAnalysis=='Ok']

In [191]:
#"Source", "Objectid", "Radicado", "Type", "Address", "Cbml", "Borough", "Comuna", "Geocod", "Desing", "Longitude", "Latitude", "X_magnamed", "Y_magnamed", "Severity", "SeverityIndex", "Date", "Year", "MonthNum", "Month", "Quarter", "Week", "Day", "WeekDayNum", "WeekDay", "Holiday", "IsHoliday", "Time", "Hour", "Motorcycle"
# i_df['Date'] = i_df.Date.dt.strftime('%d/%m/%Y')
# i_df['Time'] = i_df.Time.dt.strftime('%H:%M:%S')

# i_df['Hour']  = pd.to_datetime(i_df.Time, format='%Y-%m-%d %H:%M:%S').dt.hour
# i_df['Date'] = pd.to_datetime(i_df.Date,format='%d/%m/%Y')


Radicado          True
Latitude          True
Longitude         True
Type              True
Address          False
Cbml              True
Severity         False
Comuna           False
Borough           True
Desing            True
Year             False
Date             False
Time              True
codigo_comuna    False
Comuna           False
dtype: bool

In [263]:
for i, row in i_df[(~i_df.Latitude.isna()) & (i_df.Borough.isna())].iterrows():
    p1 = geometry.Point(float(row.Longitude), float(row.Latitude))
    borough_row = find_borough(borough_df, p1)
    
    if borough_row is not None:
        i_df.Comuna = borough_row.NOMBRE_COMUNA
        i_df.Cbml = borough_row.CODIGO
        i_df.Borough = borough_row.NOMBRE_BARRIO
i_df[(~i_df.Latitude.isna()) & (i_df.Borough.isna())]

KeyboardInterrupt: 

In [228]:
i_df[(~i_df.Latitude.isna()) & (i_df.Borough.isna())]

,Radicado,Latitude,Longitude,Type,Address,Cbml,Severity,Comuna,Borough,Desing,Year,Date,Time,codigo_comuna,Comuna
10,1423899,6.221806,-75.703816,Atropello,CL 115 CR 39 BE,NaN,Con heridos,Sin Inf,NaN,Tramo de via,2014,2014-01-01,01:40:00,Sin Inf,Sin Inf
20,1423918,6.221806,-75.703816,Caida Ocupante,CR 63 CL 49 B Sur,NaN,Con heridos,Sin Inf,NaN,Tramo de via,2014,2014-01-01,04:45:00,Sin Inf,Sin Inf
29,1423930,6.221806,-75.703816,Atropello,CR 999 CL 999,NaN,Con heridos,Sin Inf,NaN,Tramo de via,2014,2014-01-01,17:40:00,Sin Inf,Sin Inf
44,1423948,6.221806,-75.703816,Atropello,Con,NaN,Con heridos,Sin Inf,NaN,Lote o Predio,2014,2014-01-01,22:30:00,Sin Inf,Sin Inf
45,1423949,6.221806,-75.703816,Atropello,Con,NaN,Con heridos,Sin Inf,NaN,Lote o Predio,2014,2014-01-01,21:30:00,Sin Inf,Sin Inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288935,1717521,6.221806,-75.703816,Otro,CR 62 CL 70,NaN,Con heridos,Sin Inf,NaN,Tramo de via,2020,2020-06-30,10:00:00,Sin Inf,Sin Inf
288946,1717697,6.274931,-75.595161,Choque,CR 84 CL 65,NaN,Con heridos,Robledo,NaN,Interseccion,2020,2020-06-30,13:00:00,7,07 - Robledo
288954,1717550,6.221415,-75.703776,Choque,CR 64AA CL 113,NaN,Con heridos,Sin Inf,NaN,Tramo de via,2020,2020-06-30,15:00:00,Sin Inf,Sin Inf
288968,1717508,6.256059,-75.575423,Choque,CR 62 CL 53A,NaN,Con heridos,La Candelaria,NaN,Tramo de via,2020,2020-06-30,17:35:00,10,10 - La Candelaria


In [177]:
i_df['MonthNum']= i_df.Date.dt.month
i_df['Month']= i_df.Date.dt.month_name()
i_df['Quarter'] = i_df.Date.dt.quarter
i_df['Week'] = i_df.Date.dt.week
i_df['Day'] = i_df.Date.dt.day
i_df['WeekDayNum'] = i_df.Date.dt.dayofweek
i_df['WeekDay']  = i_df.Date.dt.day_name()
i_df['Hour']  = i_df.Time.hour

col_holidays = holidays.COL(years = list(range(2014,2021))) 
h_df = pd.DataFrame([col_holidays.keys(), col_holidays.values()]).T
h_df.rename(columns={0:'Date',1:'Holiday'},inplace=True)

h_df['Date']= pd.to_datetime(h_df.Date,format='%Y/%m/%d')
h_df
# 
#i_df['Holiday'] = i_df.
# i_df['WeekDay'].value_counts().sort_index()

AttributeError: 'DataFrame' object has no attribute 'Date'

In [136]:
i_df = i_df.merge(h_df,on='Date',how='left')
i_df

,Radicado,Latitude,Longitude,Type,Address,Cbml,Severity,Comuna,Borough,Desing,Year,Date,Time,codigo_comuna,Comuna,MonthNum,Month,Quarter,Week,Day,WeekDayNum,WeekDay,Hour,Holiday
0,1423828,6.219016,-75.602726,Choque,CR 80 CL 8,1611,Con heridos,Belén,Loma de los Bernal,Tramo de via,2014,2014-01-01,02:20:00,16,16 - Belén,1,January,1,1,1,2,Wednesday,0,Año Nuevo [New Year's Day]
1,1423839,6.260009,-75.568185,Atropello,CR 53 CL 61,1003,Con heridos,La Candelaria,Jesús Nazareno,Tramo de via,2014,2014-01-01,00:50:00,10,10 - La Candelaria,1,January,1,1,1,2,Wednesday,0,Año Nuevo [New Year's Day]
2,1423840,6.264765,-75.549943,Atropello,CR 39 CL 70,0308,Con heridos,Manrique,Manrique Oriental,Tramo de via,2014,2014-01-01,01:00:00,3,03 - Manrique,1,January,1,1,1,2,Wednesday,0,Año Nuevo [New Year's Day]
3,1423849,6.234327,-75.607611,Atropello,CL 32 CR 84,1617,Con heridos,Belén,Las Mercedes,Tramo de via,2014,2014-01-01,00:37:00,16,16 - Belén,1,January,1,1,1,2,Wednesday,0,Año Nuevo [New Year's Day]
4,1423890,6.300243,-75.579644,Caida Ocupante,CR 80 CL 101,0603,Con heridos,Doce De Octubre,Doce de Octubre No.2,Tramo de via,2014,2014-01-01,10:40:00,6,06 - Doce de Octubre,1,January,1,1,1,2,Wednesday,0,Año Nuevo [New Year's Day]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288978,1717678,6.220716,-75.559017,Caida Ocupante,CL 19A CR 27,9004,Con heridos,Corregimiento De Santa Elena,9004,Lote o Predio,2020,2020-06-30,21:00:00,90,90 - Corregimiento de Santa Elena,6,June,2,27,30,1,Tuesday,0,NaN
288979,1717650,6.286417,-75.570338,Choque,CR 65 CL 94,0511,Con heridos,Castilla,Castilla,Interseccion,2020,2020-06-30,21:05:00,5,05 - Castilla,6,June,2,27,30,1,Tuesday,0,NaN
288980,1717526,6.221415,-75.703776,Choque,CR 86 CL 44 B Sur,1204,Con heridos,La América,La América,Tramo de via,2020,2020-06-30,21:15:00,12,12 - La América,6,June,2,27,30,1,Tuesday,0,NaN
288981,1717545,6.235761,-75.538379,Otro,CR 12 CL 53,0813,Con heridos,Villa Hermosa,Villatina,Lote o Predio,2020,2020-06-30,22:00:00,8,08 - Villa Hermosa,6,June,2,27,30,1,Tuesday,0,NaN


In [145]:
i_df[i_df.Cbml== '03070040016']

,Radicado,Latitude,Longitude,Type,Address,Cbml,Severity,Comuna,Borough,Desing,Year,Date,Time,Comuna,MonthNum,Month,Quarter,Week,Day,WeekDayNum,WeekDay,Hour,Holiday,IsHoliday
483,00523,6.263128,-75.554245,Choque,CR 44 CL 67 - 18,03070040016,Con muertos,Manrique,Manrique Central No. 2,NaN,2014,06/01/2014,10:56:00,03 - Manrique,1,January,1,2,6,0,Monday,0,Día de los Reyes Magos [Epiphany],1


In [142]:
i_df['IsHoliday'] = i_df.Holiday.apply(lambda x : 0 if x is np.nan else 1)
i_df['Date'] = i_df.Date.dt.strftime('%d/%m/%Y')


AttributeError: Can only use .dt accessor with datetimelike values

In [143]:
i_df.drop(columns='codigo_comuna',inplace=True)

In [150]:
i_df['Hour']

0         0
1         0
2         0
3         0
4         0
         ..
288978    0
288979    0
288980    0
288981    0
288982    0
Name: Hour, Length: 288983, dtype: int64

In [149]:
# dont forget to import event
from sqlalchemy import event, create_engine
 
DB_USERNAME = 'postgres@psql-ds4a-prod'
DB_PASSWORD = 'FliFUDlbO72cq2h9AaFF'
HOST = 'psql-ds4a-prod.postgres.database.azure.com'
engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{HOST}/ds4a', connect_args={'sslmode':'require'}, max_overflow=20)


i_df.to_sql(name='incidents', con=engine,schema='processed',if_exists='append',index=False,chunksize=1000, method='multi')




```pwsh
Install-Module -Name AzureRM.profile
Install-Module -Name MSAL.PS

https://docs.microsoft.com/en-us/sharepoint/dev/sp-add-ins/add-in-permissions-in-sharepoint

https://celerik-admin.sharepoint.com/_layouts/15/appinv.aspx

<AppPermissionRequests AllowAppOnlyPolicy="true">
  <AppPermissionRequest Scope="http://sharepoint/content/sitecollection/web" Right="Read" />
  <AppPermissionRequest Scope="http://sharepoint/content/sitecollection/web/list" Right="Read"/>
</AppPermissionRequests>

https://login.microsoftonline.com/celerik.onmicrosoft.com/.well-known/openid-configuration

$tenantId = "65de2287-e076-4804-9fa6-ddbd24d2d0ff"
$authority = "https://login.microsoftonline.com/$tenantId"
$resource = "https://graph.microsoft.com"
$clientId = "f768001f-0170-455a-a3da-cdd617efe34b"
$clientSecret = '/KlFUYi9ZI0Jye0/ZY39w0uKARlZSIUHovkh6lYOjtY='
$secClientSecret = (ConvertTo-SecureString '/KlFUYi9ZI0Jye0/ZY39w0uKARlZSIUHovkh6lYOjtY=' -AsPlainText -Force)
$redirectUri = 'https://celerik.com'

$token = Get-MsalToken -clientID $clientId -clientSecret $secClientSecret -tenantID $tenantId
$token.AccessToken

$clientCreds = New-Object Microsoft.IdentityModel.Clients.ActiveDirectory.ClientCredential($clientId, $clientSecret);

$authUrl = 'https://login.microsoftonline.com/{0}/oauth2/token' -f $tenantId

$authContext = New-Object Microsoft.IdentityModel.Clients.ActiveDirectory.AuthenticationContext -ArgumentList $authUrl,$false


$authResult = $authContext.AcquireTokenAsync($resource, $clientCreds);

$authResult.Wait()

	
$authResult.Result.AccessToken

# Server name, database name and the connection string that will be used to open connection
$sqlServerUrl = "fcobo.database.windows.net"
$database = "TESTDB"
$connectionString = "Server=tcp:$sqlServerUrl,1433;Initial Catalog=$database;Persist Security Info=False;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;"

# Create the connection object
$connection = New-Object System.Data.SqlClient.SqlConnection($connectionString)

# Set AAD generated token to SQL connection token
$connection.AccessToken = $authResult.Result.AccessToken
$connection.AccessToken

# prints the connection information, it's still closed at this point, will open later on. The token is already attached.
$connection

# Query that will be sent when the connection is open. I had a 4,000 record table and I was able to truncate with this script
$query = "TRUNCATE TABLE TEST1"

# Opens connection to Azure SQL Database and executes a query
$connection.Open()
# After this, the token is no longer there, I believe this is because the authentication went through already, so it gets rid of it.
$connection
$command = New-Object -Type System.Data.SqlClient.SqlCommand($query, $connection)
$command.ExecuteNonQuery()
$connection.Close()
 

Client Secret:

	
/KlFUYi9ZI0Jye0/ZY39w0uKARlZSIUHovkh6lYOjtY=
 

Title:

	
test

App Domain:

	
www.celerik.com
	Example: "www.contoso.com"

Redirect URI:

	
https://www.celerik.com




$url = "https://login.microsoftonline.com/celerik/oauth2/token";
 $body = @{
    resource='https://celerik.sharepoint.com/sites/idlink'
    client_id='f768001f-0170-455a-a3da-cdd617efe34b'
    grant_type=code
    username={userName}
    password={password}
  }
  $json = $body | ConvertTo-Json

  ```

In [ ]:

urls = {'url_accidentalidad' : 'http://medata.gov.co/sites/default/files/medata_harvest_files/accidentalidad.csv',
'url_accidentalidad_georreferenciada_2019' : 'https://opendata.arcgis.com/datasets/e132c88e875b429db4c1a3c80b7977da_20.csv',
'url_accidentalidad_georreferenciada_2018' : 'https://opendata.arcgis.com/datasets/e4b4645cc47b462495305db0574982b5_16.csv',
'url_accidentalidad_georreferenciada_2017' : 'https://opendata.arcgis.com/datasets/b7f7158734f64e7f849df6ac4a6b10f1_12.csv',
'url_accidentalidad_georreferenciada_2016' : 'https://opendata.arcgis.com/datasets/a65c3aff0ef34973a2441b6cd0fbc24a_8.csv',
'url_accidentalidad_georreferenciada_2015' : 'https://opendata.arcgis.com/datasets/025f3be66fbd48b888b779cf00928ae8_4.csv',
'url_accidentalidad_georreferenciada_2014' : 'https://opendata.arcgis.com/datasets/505e89d2ade143a684d51b60236ba285_0.csv',
'url_accidentalidad_motos':'https://www.datos.gov.co/api/views/b7ik-2upt/rows.csv?accessType=DOWNLOAD',
'url_camaras_cctv' : 'https://opendata.arcgis.com/datasets/9721295ac3644bfbb076795b36912037_9.csv',
'url_camaras_fotodeteccion' : 'https://opendata.arcgis.com/datasets/bfcd5e4e3e364957b1aeeff0ee022815_7.csv',
'url_cruces_semafóricos' : 'https://opendata.arcgis.com/datasets/072ed4e620b849d6bdbb058c3cb77922_0.csv',
'url_limite_comunas' : 'https://opendata.arcgis.com/datasets/283d1d14584641c9971edbd2f695e502_6.csv',
'url_limite_barrios' : 'https://opendata.arcgis.com/datasets/1a6dbf15865b4357aa559699ea90c5b9_7.csv'}

for key in urls:
    r =requests.get(urls[key], allow_redirects=True)
    open(key.replace('url_','')+'.csv', 'wb').write(r.content)

### Concatenate all the accidentalidad_georreferenciada files

In [149]:
# ag_df: yearly file
ag_df = pd.DataFrame()
path = "./"
for  file in os.listdir(path):
    if 'accidentalidad_georreferenciada_' in file:
        df_tmp = pd.read_csv(os.path.join(path, file))
        df_tmp.columns = [c.lower() for c in df_tmp.columns]
        df_tmp['file'] = file
        ag_df = pd.concat([ag_df, df_tmp])

ag_df.to_csv("../raw/accidentalidad_georreferenciada.csv",index=False)

### Check the columns

# Delete 'radicados' duplicates from three sources:
- accidentalidad.csv
- accidentalidad_georreferenciada.csv
- accidentalidad_motos.csv

We found some duplicates values in "radicado" field so we procedure to drop them.


In [58]:
ag_df = pd.read_csv("../raw/accidentalidad_georreferenciada.csv")

a_df = pd.read_csv('./accidentalidad.csv', skiprows=1,header=None,sep=';')
a_df.rename(columns={18:'fecha_hora',22: 'objectid', 23:'radicado'},inplace=True)

am_df = pd.read_csv('./accidentalidad_motos.csv')
am_df['moto']=1






In [66]:
ag_df = ag_df.sort_values(by=["radicado", "objectid"])
ag_df.head(5)

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
93458,-75.591921,6.282942,507560,12,2017/01/01 00:00:00+00,05:05:00,1,2017,Atropello,CL 77 A CR 88,CL 077 A 088 000 00000,0713,Malla vial,MUERTO,Aures No.1,Robledo,NaN,DOMINGO,1,NaN,832386.43,1186760.60,-75.591921,6.282942,accidentalidad_georreferenciada_2017.csv,NaN
93459,-75.613491,6.284049,507561,20,2017/01/01 00:00:00+00,08:30:00,1,2017,Choque,CL 66 CR 103 B - 15,CL 066 103 B 015 00000,60980380285,Nomenclatura con Interior,MUERTO,Área de Expansión Pajarito,Corregimiento de San Cristóbal,NaN,DOMINGO,1,NaN,829999.08,1186883.23,-75.613491,6.284049,accidentalidad_georreferenciada_2017.csv,NaN
223849,-75.604448,6.250820,252567,38,2014/11/25 00:00:00+00,12:33:00,25,2014,Atropello,CR 84 CL 44 - 05,CR 084 044 005 00000,1204,Malla vial,MUERTO,La América,La América,NaN,MARTES,11,NaN,830999.75,1183207.50,-75.604448,6.250820,accidentalidad_georreferenciada_2014.csv,NaN
93460,-75.703816,6.221806,507562,47,2017/01/02 00:00:00+00,01:30:00,2,2017,Choque,CR 42 CL 92 - 50,CR 042 092 050 00000,NaN,No Ubicada,MUERTO,NaN,NaN,NaN,LUNES,1,NaN,820000.00,1180000.00,-75.703816,6.221806,accidentalidad_georreferenciada_2017.csv,NaN
93462,-75.567295,6.254069,507564,499,2017/01/05 00:00:00+00,07:55:00,5,2017,Atropello,CR 51 D CL 54,CR 051 D 054 000 00000,1005,Malla vial aproximada: CR 51-54,MUERTO,Estación Villa,La Candelaria,NaN,JUEVES,1,NaN,835112.08,1183566.79,-75.567295,6.254069,accidentalidad_georreferenciada_2017.csv,NaN


In [45]:
ag_df[ag_df['radicado']=='62196']

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
132131,-75.557246,6.299648,546233,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN
132132,-75.557246,6.299648,546234,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN
132133,-75.557246,6.299648,546235,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN


In [47]:
ag_df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)

In [49]:
ag_df[ag_df['radicado']=='62196']

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
132131,-75.557246,6.299648,546233,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN


In [52]:
ag_df['gravedad'].value_counts()

HERIDO         144059
SOLO DAÑOS     116379
Con heridos      7163
Solo daños       5269
MUERTO           1530
Con muertos        61
Name: gravedad, dtype: int64

In [59]:
ag_df[ag_df['objectid']==507563]


,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
93461,-75.559541,6.245486,507563,5.0016e+19,2017/01/03 00:00:00+00,09:41:00,3,2017,Choque y Atropello,CR 39 CL 50,CR 039 050 000 00000,1016,Malla vial,MUERTO,Boston,La Candelaria,NaN,MARTES,1,NaN,835970.4,1182617.38,-75.559541,6.245486,accidentalidad_georreferenciada_2017.csv,NaN


In [ ]:
df.clase = df.clase.apply(lambda x: unidecode(x).upper().strip().replace(' de ',' '))

In [27]:
from ast import literal_eval
[l.lower() for l in list(literal_eval(list(a_df.loc[a_df[0]=="1.0", 4][0:1])[0]).keys())]

['@timestamp',
 'cbml',
 'expediente',
 'tipo',
 'direccion encasillada',
 'clase_accidente',
 'location',
 'mes',
 'fecha_accidentes',
 'año',
 'comuna',
 'tags',
 'direccion',
 'host',
 'fingerprint',
 'path',
 'numcomuna',
 'diseño',
 'barrio',
 'gravedad_accidente',
 'fecha_accidente',
 'y',
 'x',
 '@version',
 'id',
 'nro_radicado']

In [39]:
a_df = a_df.sort_values(by=['radicado', 'objectid'])
a_df['radicado'].value_counts()


1613593    1
1620992    1
1645994    1
1567028    1
1465342    1
          ..
1451850    1
1442267    1
1635939    1
1436460    1
1466846    1
Name: radicado, Length: 261875, dtype: int64

In [29]:
a_df.drop_duplicates(subset=['radicado'], keep="first", inplace=True)

In [30]:
am_df = am_df.sort_values(by=['NRO_RADICADO'])
am_df['NRO_RADICADO'].value_counts()

1647286    2
1638618    2
1641667    2
1647440    2
1640023    2
          ..
1578967    1
1603547    1
1593308    1
1591261    1
1574913    1
Name: NRO_RADICADO, Length: 136653, dtype: int64

In [31]:
ag_df = ag_df[~ag_df.radicado.isna()]
a_df=a_df[~a_df.radicado.isna()]
dtype = dict(radicado=str)
ag_df.radicado = ag_df.radicado.astype(dtype)
a_df.radicado = a_df.radicado.astype(dtype)
ag_df.radicado = ag_df.radicado.str.strip()
ag_df.radicado = ag_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
ag_df.radicado = ag_df.radicado.str.replace("^(0+)(\d+)", "\\2")
a_df.radicado = a_df.radicado.str.strip()
a_df.radicado = a_df.radicado.str.replace("(\d+)(.0)*","\\1")

In [32]:
am_df.drop_duplicates(subset=['NRO_RADICADO'], keep="first", inplace=True)

Merge accidentes geo with accidentes, and motos

In [33]:
ag_df = ag_df.merge(a_df[['fecha_hora','radicado']],on='radicado',how='left')

In [34]:
am_df.rename(columns={'NRO_RADICADO':'radicado'},inplace=True)

ag_df = ag_df.merge(am_df[['radicado','moto']],on='radicado',how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [155]:
import numpy as np
ag_df['fecha'] = ag_df['fecha'].apply(lambda x : x.replace(' 00:00:00+00',''))
ag_df['fecha'] = pd.to_datetime(ag_df['fecha'],format='%Y/%m/%d')

ag_df[ag_df['fecha'].dt.year>2014]['moto'].fillna(0,inplace=True)



In [156]:
ag_df[ag_df['radicado'].isin(am_df['radicado'])].groupby(pd.Grouper(key='fecha',freq='Y')).count()

,x,y,objectid,radicado,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape,fecha_hora,moto
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31,19441,19441,19441,19441,19441,19441,19441,19441,19441,19240,17471,19441,19441,17471,17471,19441,19441,19441,0,19441,19441,19441,19441,19441,0,0,19441
2016-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-12-31,24596,24596,24596,24596,24596,24596,24596,24596,24596,24517,22701,24596,24596,22692,22701,24596,24596,24596,0,24596,24596,24596,24596,24596,0,0,24596
2019-12-31,19246,19246,26804,26804,26804,26804,26804,26804,26804,26765,25216,26804,26804,24718,25915,26804,26804,26804,26804,26804,26804,26804,26804,26804,0,4271,26804


## How did you merge the two DataFrames (aggregated data from 2014-2019 and accidentalidad.csv)

In [28]:
print(a_df.isna().any())
print(ag_df.isna().any())
print(am_df.isna().any())


0           False
1            True
2            True
3           False
4            True
5            True
6            True
7            True
8            True
9            True
10           True
11           True
12           True
13           True
14           True
15           True
16           True
17           True
fecha        True
19           True
20           True
21           True
22           True
radicado     True
dtype: bool
x                 True
y                 True
objectid         False
radicado          True
fecha            False
hora             False
dia              False
periodo          False
clase             True
direccion        False
direccion_enc     True
cbml              True
tipo_geocod      False
gravedad         False
barrio            True
comuna            True
diseno            True
dia_nombre       False
mes              False
mes_nombre        True
x_magnamed       False
y_magnamed       False
longitud         False
latitud          False
file

In [112]:
ag_df[ag_df.radicado.isna()] # check the nulls

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
39496,-75.561594,6.293715,546224,NaN,2017/12/21 00:00:00+00,09:58:00,21,2017,Atropello,CR 52 CL 100 - 75,CR 052 100 075 00000,02100090113,Nomenclatura con Interior,MUERTO,Santa Cruz,Santa Cruz,NaN,JUEVES,12,NaN,835743.14,1187952.15,-75.561594,6.293715,Accidentalidadgeorreferenciada2017.csv,NaN
40150,-75.578129,6.247262,546941,NaN,2017/09/06 00:00:00+00,05:45:00,6,2017,Atropello,CL 44 entre CR 59 y 60,"CL 044 59, 060 00000",1008,Malla vial,MUERTO,La Candelaria,Corazón de Jesús,NaN,MIÉRCOLES,9,NaN,833912.85,1182813.77,-75.578129,6.247262,Accidentalidadgeorreferenciada2017.csv,NaN
37075,-75.564198,6.261494,683301,NaN,2018/05/07 00:00:00+00,06:00:00,7,2018,Choque,CR 51 CL 64,CR 051 064 000 00000,1001,Malla vial,MUERTO,Prado,La Candelaria,NaN,LUNES,5,NaN,835454.94,1184388.09,-75.564198,6.261494,Accidentalidadgeorreferenciada2018.csv,NaN
37076,-75.546095,6.271341,683302,NaN,2018/11/01 00:00:00+00,07:40:00,1,2018,Choque,CL 80 CR 32 - 82,CL 080 032 082 00000,03040160010,Catastro con Interior,MUERTO,Santa Inés,Manrique,NaN,JUEVES,11,NaN,837458.56,1185477.35,-75.546095,6.271341,Accidentalidadgeorreferenciada2018.csv,NaN
37077,-75.593598,6.270184,683303,NaN,2018/12/26 00:00:00+00,12:30:00,26,2018,Choque,CL 59 CR 78 AA,CL 059 078 AA 000 00000,1114,Malla vial,MUERTO,Los Colores,Laureles Estadio,NaN,MIÉRCOLES,12,NaN,832200.75,1185349.38,-75.593598,6.270184,Accidentalidadgeorreferenciada2018.csv,NaN


In [113]:
ag_df.clase.value_counts(dropna=False)

Choque                170276
Otro                   26585
Atropello              24647
Caida Ocupante         16419
Volcamiento             8249
Caída de Ocupante       5665
Incendio                  32
Caída Ocupante            16
NaN                        6
Choque                     3
Choque y Atropello         1
Name: clase, dtype: int64

In [114]:
ag_df.cbml.value_counts(dropna=False)

1019           6182
0517           5246
1507           5153
1012           4988
1105           4461
               ... 
80004340062       1
14220240062       1
15100350002       1
02030120001       1
11140030016       1
Name: cbml, Length: 712, dtype: int64

In [115]:
ag_df.radicado.value_counts()[0:21]

62196                    3
1618580.0                2
39226                    2
1662064                  2
1640023.0                2
31435                    2
1656231                  2
1639346.0                2
1639954.0                2
58560                    2
1647440.0                2
58864                    2
1647286.0                2
1655403                  2
48996                    2
050016000206201523690    2
1640118.0                2
1640922.0                2
1640356.0                2
1638618.0                2
1689154                  1
Name: radicado, dtype: int64

In [116]:
ag_df[ag_df.radicado==1638618.0]

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
21533,-75.585424,6.241112,666622,1.63862e+06,2018/08/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,MARTES,8,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN
36260,-75.585424,6.241112,682426,1.63862e+06,2018/10/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,DOMINGO,10,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN


In [117]:
a_df.radicado.value_counts(dropna=False)[0:30]

NaN          7
1687153.0    3
1687280.0    3
1689208.0    3
1687560.0    3
1687158.0    3
1687159.0    3
1687162.0    3
1687163.0    3
62196        3
1687164.0    3
1687165.0    3
1689064.0    3
1687166.0    3
1685159.0    3
1687167.0    3
1687169.0    3
1687154.0    3
1685140.0    3
1687171.0    3
1687292.0    3
1685517.0    3
1687136.0    3
1685493.0    3
1687138.0    3
1687557.0    3
1684827.0    3
1687140.0    3
1687144.0    3
1688636.0    3
Name: radicado, dtype: int64

In [118]:
df[df.radicado=="1638618"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
205150,2019-06-04T20:55:57.306Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",8.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-08-14T22:35:00.000Z,1182133.61,833105.42,1.0,211762.0,1638618
213113,2019-06-04T20:56:23.935Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",10.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-10-14T22:35:00.000Z,1182133.61,833105.42,1.0,219306.0,1638618


In [119]:
df[df.radicado=="1638617"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
203433,2019-06-04T20:55:58.641Z,402,A000846480,Malla vial,CR 051 092 000 00000,Choque,"[-75.5611117649, 6.28348983493]",8.0,2018.0,Aranjuez,[u'incidentes'],CR 51 CL 92,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,04,Tramo de via,San Isidro,Solo daños,2018-08-15T20:00:00.000Z,1186820.47,835813.52,1.0,211882.0,1638617


We found some duplicates values in "radicado" field so we procedure to drop them.

In [120]:
df = df.sort_values(by=["radicado", "id"])
df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)

In order to have date and time in different columns we decided to derivate new columns from "fecha" and have separet columns for the date and the hour.

In [121]:
df["fecha_incidente"] = df.fecha.str[0:10]
df["hora_incidente"] = df.fecha.str[11:13]

In [122]:
# drop duplicates by "radicado", "objectid"
agg_df = agg_df.sort_values(by=["radicado", "objectid"])
agg_df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)
agg_df = agg_df[(~agg_df.radicado.isna()) & (agg_df.radicado != "0")]

In [123]:
df.radicado

262153    1.67637e+06
262150    1.67644e+06
262168    1.67646e+06
262162    1.67646e+06
262160    1.67647e+06
             ...     
143894           9769
9909             9778
12090            9780
183864           9982
172418            NaN
Name: radicado, Length: 261876, dtype: object

In [124]:
df[df.radicado.isna()] # check the number of nulls

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
172418,2019-06-04T20:54:26.357Z,1008,A000665813,Malla vial,"CL 044 59, 060 00000",Atropello,"[-75.5781789259, 6.24722232125]",9.0,2017.0,La Candelaria,[u'incidentes'],CL 44 entre CR 59 y 60,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,10,NaN,Corazón de Jesús,Con muertos,2017-09-06T10:45:00.000Z,1182813.77,833912.85,1.0,185160.0,NaN,2017-09-06,10


In [125]:
agg_df = agg_df[~agg_df.radicado.isna()]
df=df[~df.radicado.isna()]
dtype = dict(radicado=str)
agg_df.radicado = agg_df.radicado.astype(dtype)
df.radicado = df.radicado.astype(dtype)
agg_df.radicado = agg_df.radicado.str.strip()
agg_df.radicado = agg_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
agg_df.radicado = agg_df.radicado.str.replace("^(0+)(\d+)", "\\2")
df.radicado = df.radicado.str.strip()
df.radicado = df.radicado.str.replace("(\d+)(.0)*","\\1")

In [126]:
df.fecha.max()

'2019-10-01T04:20:00.000Z'

In [131]:
df.shape
df
# r = df[(df.cbml.isna()) & (df.x_long_y_lat == '[-75.7037762763, 6.22141524356]') ][['x_long_y_lat','barrio','comuna','direccion','direccion_enc','cbml']]
# r

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
262153,2019-11-07T16:07:39.311Z,103,A001002056,Malla vial,CR 043 A 110 B 000 00000,Atropello,"[-75.5478922382, 6.29966416516]",6.0,2019.0,Popular,[u'incidentes'],CR 43 A CL 110 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,01,Tramo de via,Popular,Con heridos,2019-07-01T02:50:00.000Z,1.188606e+06,837281.840000,1.0,10647.0,1676369,2019-07-01,02
262150,2019-11-07T16:07:39.311Z,1102,A001002214,Malla vial,CR 066 048 D 000 00000,Choque,"[-75.5829966564, 6.25459513351]",6.0,2019.0,Laureles Estadio,[u'incidentes'],CR 66 CL 48 D,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Lote o Predio,Suramericana,Solo daños,2019-07-01T01:40:00.000Z,1.183631e+06,833381.860000,1.0,10641.0,1676437,2019-07-01,01
262168,2019-11-07T16:07:39.312Z,607,A001002310,Malla vial,CR 083 091 A 000 00000,Choque,"[-75.5849391248, 6.28861482868]",7.0,2019.0,Doce de Octubre,[u'incidentes'],CR 83 CL 91 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,Tramo de via,Kennedy,Solo daños,2019-07-01T23:20:00.000Z,1.187395e+06,833177.650000,1.0,10699.0,1676456,2019-07-01,23
262162,2019-11-07T16:07:39.312Z,312,A001002302,Malla vial,CR 023 B 084 B 000 00000,Choque,"[-75.5378038865, 6.27686386215]",7.0,2019.0,Manrique,[u'incidentes'],CR 23 B CL 84 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,Tramo de via,Oriente,Solo daños,2019-07-01T20:30:00.000Z,1.186080e+06,838391.450000,1.0,10686.0,1676461,2019-07-01,20
262160,2019-11-07T16:07:39.312Z,NaN,A001002169,No Ubicada,CR 067 025 000 00000,Choque,"[-75.7037762763, 6.22141524356]",7.0,2019.0,Sin Inf,[u'incidentes'],CR 67 CL 25,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,No ubicada,Tramo de via,NaN,Con heridos,2019-07-01T16:40:00.000Z,1.180000e+06,820000.000000,1.0,10674.0,1676468,2019-07-01,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2019-06-04T20:44:08.013Z,3.02021e+09,A1485619,Catastro con Interior,CR 039 085 A 035 00000,Choque,"[-75.5477362391, 6.27745474338]",2.0,2014.0,Manrique,[u'incidentes'],CR 39 CL 85 A - 35,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,NaN,Las Granjas,Con muertos,2014-02-24T08:03:00.000Z,1.186149e+06,837292.200000,1.0,6512.0,9766,2014-02-24,08
143894,2019-06-04T20:53:18.109Z,6.08101e+09,A000497560,Catastro con Interior,CR 083 101 A 021 00000,Atropello,"[-75.5821690996, 6.30114194037]",2.0,2017.0,Doce de Octubre,[u'incidentes'],CR 83 CL 101 A - 21,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,NaN,Picacho,Con muertos,2017-02-12T01:30:00.000Z,1.188780e+06,833488.260000,1.0,165498.0,9769,2017-02-12,01
9909,2019-06-04T20:44:08.014Z,1.50304e+10,A1485792,Catastro con Interior,CL 025 065 C 050 00000,Atropello,"[-75.5855316677, 6.22630141567]",2.0,2014.0,Guayabal,[u'incidentes'],CL 25 CR 65 C - 50,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,15,NaN,Trinidad,Con muertos,2014-02-24T12:30:00.000Z,1.180502e+06,833092.280000,1.0,6513.0,9778,2014-02-24,12
12090,2019-06-04T20:44:08.014Z,1.62002e+10,A1485790,Nomenclatura con Interior,CL 033 078 177 00000,Atropello,"[-75.6000288989, 6.23861225192]",2.0,2014.0,Belén,[u'incidentes'],CL 33 CR 78 - 177,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,NaN,El Nogal-Los Almendros,Con muertos,2014-02-24T12:15:00.000Z,1.181868e+06,831491.290000,1.0,6515.0,9780,2014-02-24,12


In [128]:
df.clase = df.clase.apply(lambda x: unidecode(x).lower().strip().replace(' de ',' '))

AttributeError: 'float' object has no attribute 'encode'

In [129]:
df.to_csv("../processed/preprocessed_accidents.csv",index=False)